We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [ ]:
import duckdb
import pandas as pd
from pathlib import Path

cwd = Path.cwd()

root = None
for p in [cwd] + list(cwd.parents):
    if (p / "db").exists():
        root = p
        break

if root is None:
    raise FileNotFoundError("Could not find a 'db' folder above the current working directory")

DB_PATH = root / "db" / "nflpa.duckdb"
print("Using DB_PATH", DB_PATH)

con = duckdb.connect(str(DB_PATH))

con.execute("PRAGMA threads=4")
con.execute("PRAGMA memory_limit='4GB'")

We verify that the primary panel aligns with the expected regular-season schedule and row counts across all seasons

In [ ]:
required = ["pbp", "schedules", "team_week_panel"]
existing = set(con.execute("SHOW TABLES").df()["name"].tolist())

missing = [t for t in required if t not in existing]

print("Missing", missing)
print("OK" if not missing else "STOP, fix missing tables before continuing")

We inspect the play-by-play schema to record which optional columns exist allowing subsequent logic to adapt safely without assuming column presence

In [ ]:
pbp_cols_df = con.execute("DESCRIBE pbp").df()
pbp_cols = set(pbp_cols_df["column_name"].tolist())

optional_flags = {
    "return_team": "return_team" in pbp_cols,
    "kickoff_type": "kickoff_type" in pbp_cols,
    "onside_kick": "onside_kick" in pbp_cols,
    "safety_kick": "safety_kick" in pbp_cols,
    "free_kick": "free_kick" in pbp_cols,
    "desc": "desc" in pbp_cols,
}

print("Optional columns present", optional_flags)

pbp_cols_df

We check that team identifiers in the panel align with those in the play-by-play data, ensuring join keys are consistent across datasets to prevent failed matches

In [ ]:
panel_cols_df = con.execute("DESCRIBE team_week_panel").df()
panel_cols = set(panel_cols_df["column_name"].tolist())

if "team_id" in panel_cols:
    PANEL_TEAM_COL = "team_id"
elif "team" in panel_cols:
    PANEL_TEAM_COL = "team"
else:
    raise ValueError(f"Could not find a team column in team_week_panel, columns are {sorted(panel_cols)[:50]}")

print("Using panel team column", PANEL_TEAM_COL)

panel_teams = set(
    con.execute(f"SELECT DISTINCT {PANEL_TEAM_COL} AS team_key FROM team_week_panel WHERE {PANEL_TEAM_COL} IS NOT NULL")
      .df()["team_key"]
      .tolist()
)

pbp_posteams = set(
    con.execute("SELECT DISTINCT posteam FROM pbp WHERE posteam IS NOT NULL")
      .df()["posteam"]
      .tolist()
)

intersect = panel_teams.intersection(pbp_posteams)

print("Distinct panel teams", len(panel_teams))
print("Distinct posteam in pbp", len(pbp_posteams))
print("Intersection size", len(intersect))

print("Sample posteam not in panel", sorted(list(pbp_posteams - panel_teams))[:25])
print("Sample panel team not in pbp", sorted(list(panel_teams - pbp_posteams))[:25])


Quick sanity check to confirm that we are not accidentally pulling older schedules seasons that include those codes

In [ ]:
con.execute(f"""
SELECT
  {PANEL_TEAM_COL} AS team,
  MIN(season) AS min_season,
  MAX(season) AS max_season,
  COUNT(*) AS rows
FROM team_week_panel
WHERE {PANEL_TEAM_COL} IN ('OAK','SD','STL')
GROUP BY 1
ORDER BY 1
""").df()

We create a minimal special teams base view using safe NULL placeholders for missing optional columns, ensuring downstream SQL remains stable and preventing schema-related breakages

In [ ]:
select_return_team = "return_team" if optional_flags["return_team"] else "NULL::VARCHAR AS return_team"
select_kickoff_type = "kickoff_type" if optional_flags["kickoff_type"] else "NULL::VARCHAR AS kickoff_type"
select_onside_kick = "onside_kick" if optional_flags["onside_kick"] else "NULL::INTEGER AS onside_kick"
select_safety_kick = "safety_kick" if optional_flags["safety_kick"] else "NULL::INTEGER AS safety_kick"
select_free_kick = "free_kick" if optional_flags["free_kick"] else "NULL::INTEGER AS free_kick"

select_desc = "\"desc\" AS play_desc" if optional_flags["desc"] else "NULL::VARCHAR AS play_desc"

con.execute(f"""
CREATE OR REPLACE TEMP VIEW pbp_st_base AS
SELECT
  season,
  week,
  game_id,
  posteam,
  defteam,
  {select_return_team},
  CASE
    WHEN play_type IN ('punt','kickoff') THEN COALESCE(return_team, defteam)
    ELSE return_team
  END AS return_team_filled,
  play_type,
  {select_kickoff_type},
  {select_onside_kick},
  {select_safety_kick},
  {select_free_kick},
  {select_desc}
FROM pbp
""")

We filter the play-by-play data to include only regular-season games, ensuring the metric calculations align with the existing panel scope.

In [ ]:
con.execute("""
CREATE OR REPLACE TEMP VIEW pbp_st_reg AS
SELECT b.*
FROM pbp_st_base b
JOIN schedules s
  ON b.game_id = s.game_id
WHERE s.game_type = 'REG'
""")

Quick sanity check to confirm that return team fields are populated often enough for punts and kickoffs and to quantify how many return events may be missing

In [ ]:
con.execute("""
SELECT
  play_type,
  COUNT(*) AS plays,
  SUM(CASE WHEN return_team IS NULL THEN 1 ELSE 0 END) AS null_return_team,
  AVG(CASE WHEN return_team IS NULL THEN 1.0 ELSE 0.0 END) AS null_return_team_rate
FROM pbp_st_reg
WHERE play_type IN ('punt', 'kickoff')
GROUP BY play_type
ORDER BY play_type
""").df()

We build a rare play condition that utilizes explicit kick flags when available and falls back to description keywords when necessary, ensuring robust identification of special teams events across varying data qualities

In [ ]:
rare_parts = []

rare_parts.append("kickoff_type IN ('onside', 'safety', 'free_kick', 'free kick', 'safety kick')")
rare_parts.append("onside_kick = 1")
rare_parts.append("safety_kick = 1")
rare_parts.append("free_kick = 1")

if optional_flags["desc"]:
    rare_parts.append("play_desc ILIKE '%onside%'")
    rare_parts.append("play_desc ILIKE '%free kick%'")
    rare_parts.append("play_desc ILIKE '%safety kick%'")
    rare_parts.append("play_desc ILIKE '%fair catch kick%'")
    rare_parts.append("play_desc ILIKE '%free-kick%'")
    rare_parts.append("play_desc ILIKE '%safety-kick%'")

rare_condition = "(" + " OR ".join(rare_parts) + ")"

print("Rare condition SQL")
print(rare_condition)

We create a long events table where each row represents a single special teams event attributed to the correct team for each play component by establishing a granular foundation for team-week aggregation

In [ ]:
con.execute(f"""
CREATE OR REPLACE TEMP VIEW st_events_long AS

SELECT
  season,
  week,
  posteam AS team_id,
  'punt' AS component
FROM pbp_st_reg
WHERE play_type = 'punt'
  AND posteam IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  return_team_filled AS team_id,
  'punt_return' AS component
FROM pbp_st_reg
WHERE play_type = 'punt'
  AND return_team_filled IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  posteam AS team_id,
  'kickoff' AS component
FROM pbp_st_reg
WHERE play_type = 'kickoff'
  AND posteam IS NOT NULL
  AND NOT {rare_condition}

UNION ALL

SELECT
  season,
  week,
  return_team_filled AS team_id,
  'kick_return' AS component
FROM pbp_st_reg
WHERE play_type = 'kickoff'
  AND return_team_filled IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  posteam AS team_id,
  'fg' AS component
FROM pbp_st_reg
WHERE play_type = 'field_goal'
  AND posteam IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  posteam AS team_id,
  'xp' AS component
FROM pbp_st_reg
WHERE play_type = 'extra_point'
  AND posteam IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  posteam AS team_id,
  'rare' AS component
FROM pbp_st_reg
WHERE play_type = 'kickoff'
  AND posteam IS NOT NULL
  AND {rare_condition}
""")

We aggregate the long events table into the required component columns for each team-week by transforming granular special teams data into a structured feature set ready for panel integration

In [ ]:
con.execute("""
CREATE OR REPLACE TEMP VIEW st_components_team_week AS
SELECT
  season,
  week,
  team_id,

  SUM(CASE WHEN component = 'punt' THEN 1 ELSE 0 END) AS ST_Punt_w,
  SUM(CASE WHEN component = 'punt_return' THEN 1 ELSE 0 END) AS ST_PuntReturn_w,
  SUM(CASE WHEN component = 'kickoff' THEN 1 ELSE 0 END) AS ST_Kickoff_w,
  SUM(CASE WHEN component = 'kick_return' THEN 1 ELSE 0 END) AS ST_KickReturn_w,
  SUM(CASE WHEN component = 'fg' THEN 1 ELSE 0 END) AS ST_FG_w,
  SUM(CASE WHEN component = 'xp' THEN 1 ELSE 0 END) AS ST_XP_w,
  SUM(CASE WHEN component = 'rare' THEN 1 ELSE 0 END) AS ST_Rare_w
FROM st_events_long
GROUP BY season, week, team_id
""")

We compute workload totals exactly as defined in the workflow while keeping all component columns for storage in the panel

In [ ]:
con.execute("""
CREATE OR REPLACE TEMP VIEW st_load_team_week AS
SELECT
  season,
  week,
  team_id,

  ST_Punt_w,
  ST_PuntReturn_w,
  ST_Kickoff_w,
  ST_KickReturn_w,
  ST_FG_w,
  ST_XP_w,
  ST_Rare_w,

  (ST_Punt_w + ST_PuntReturn_w + ST_Kickoff_w + ST_KickReturn_w + ST_FG_w + ST_XP_w + ST_Rare_w) AS ST_Load_All_w,

  (ST_Kickoff_w + ST_XP_w + ST_FG_w) AS ST_Load_ScoreLinked_w,

  (ST_Punt_w + ST_PuntReturn_w + ST_KickReturn_w + ST_Rare_w) AS ST_Load_NonScore_w
FROM st_components_team_week
""")

Quick sanity check to confirm that return team fields are populated often enough for punts and kickoffs and to quantify how many return events may be missing

In [ ]:
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT season || '-' || week || '-' || team_id) AS distinct_team_weeks,
  MIN(ST_Load_All_w) AS min_all,
  MAX(ST_Load_All_w) AS max_all,
  AVG(ST_Load_All_w) AS avg_all,
  SUM(ST_Rare_w) AS total_rare,
  MAX(ST_Rare_w) AS max_rare
FROM st_load_team_week
""").df()

Quick sanity check to confirm that return team fields are populated often enough for punts and kickoffs and to quantify how many return events may be missing

In [ ]:
con.execute("""
SELECT
  season,
  week,
  SUM(ST_Kickoff_w) AS total_kickoffs_by_kicking_team,
  SUM(ST_KickReturn_w) AS total_kick_returns_by_return_team,
  SUM(ST_Kickoff_w) - SUM(ST_KickReturn_w) AS kickoff_minus_return
FROM st_load_team_week
GROUP BY season, week
ORDER BY ABS(SUM(ST_Kickoff_w) - SUM(ST_KickReturn_w)) DESC
LIMIT 25
""").df()

We left join special teams workload components and totals into the team-week panel while preserving all existing rows and filling missing values with zeros

In [ ]:
con.execute(f"""
CREATE OR REPLACE TABLE team_week_panel AS
SELECT
  p.*,

  COALESCE(st.ST_Punt_w, 0) AS ST_Punt_w,
  COALESCE(st.ST_PuntReturn_w, 0) AS ST_PuntReturn_w,
  COALESCE(st.ST_Kickoff_w, 0) AS ST_Kickoff_w,
  COALESCE(st.ST_KickReturn_w, 0) AS ST_KickReturn_w,
  COALESCE(st.ST_FG_w, 0) AS ST_FG_w,
  COALESCE(st.ST_XP_w, 0) AS ST_XP_w,
  COALESCE(st.ST_Rare_w, 0) AS ST_Rare_w,

  COALESCE(st.ST_Load_All_w, 0) AS ST_Load_All_w,
  COALESCE(st.ST_Load_ScoreLinked_w, 0) AS ST_Load_ScoreLinked_w,
  COALESCE(st.ST_Load_NonScore_w, 0) AS ST_Load_NonScore_w

FROM team_week_panel p
LEFT JOIN st_load_team_week st
  ON p.season = st.season
 AND p.week = st.week
 AND p.{PANEL_TEAM_COL} = st.team_id
""")

Quick sanity check to confirm row counts are unchanged and verify that workload columns are populated across the full panel

In [ ]:
con.execute(f"""
SELECT
  COUNT(*) AS panel_rows,
  COUNT(DISTINCT season || '-' || week || '-' || {PANEL_TEAM_COL}) AS distinct_team_weeks,
  SUM(ST_Load_All_w) AS sum_all,
  SUM(ST_Load_ScoreLinked_w) AS sum_scorelinked,
  SUM(ST_Load_NonScore_w) AS sum_nonscore
FROM team_week_panel
""").df()

We compute distribution summaries for each component and total to catch overcounting or misaggregation quickly

In [ ]:
con.execute("""
SELECT
  MIN(ST_Punt_w) AS min_punt,
  MAX(ST_Punt_w) AS max_punt,
  AVG(ST_Punt_w) AS avg_punt,

  MIN(ST_PuntReturn_w) AS min_puntret,
  MAX(ST_PuntReturn_w) AS max_puntret,
  AVG(ST_PuntReturn_w) AS avg_puntret,

  MIN(ST_Kickoff_w) AS min_kickoff,
  MAX(ST_Kickoff_w) AS max_kickoff,
  AVG(ST_Kickoff_w) AS avg_kickoff,

  MIN(ST_KickReturn_w) AS min_kickret,
  MAX(ST_KickReturn_w) AS max_kickret,
  AVG(ST_KickReturn_w) AS avg_kickret,

  MIN(ST_FG_w) AS min_fg,
  MAX(ST_FG_w) AS max_fg,
  AVG(ST_FG_w) AS avg_fg,

  MIN(ST_XP_w) AS min_xp,
  MAX(ST_XP_w) AS max_xp,
  AVG(ST_XP_w) AS avg_xp,

  MIN(ST_Rare_w) AS min_rare,
  MAX(ST_Rare_w) AS max_rare,
  AVG(ST_Rare_w) AS avg_rare,

  MIN(ST_Load_All_w) AS min_all,
  MAX(ST_Load_All_w) AS max_all,
  AVG(ST_Load_All_w) AS avg_all
FROM team_week_panel
""").df()

Quick sanity check to confirm that return team fields are populated often enough for punts and kickoffs and to quantify how many return events may be missing

In [ ]:
con.execute(f"""
SELECT
  season,
  week,
  {PANEL_TEAM_COL} AS team,
  ST_Punt_w,
  ST_PuntReturn_w,
  ST_Kickoff_w,
  ST_KickReturn_w,
  ST_FG_w,
  ST_XP_w,
  ST_Rare_w,
  ST_Load_All_w,
  ST_Load_ScoreLinked_w,
  ST_Load_NonScore_w
FROM team_week_panel
ORDER BY ST_Load_All_w DESC, season DESC, week DESC, {PANEL_TEAM_COL}
LIMIT 50
""").df()

Quick sanity check to confirm that return team fields are populated often enough for punts and kickoffs and to quantify how many return events may be missing

In [ ]:
con.execute("""
SELECT
  season,
  week,
  team,
  ST_Load_All_w,
  ST_Load_ScoreLinked_w,
  ST_Load_NonScore_w
FROM team_week_panel
ORDER BY season DESC, week DESC, team
LIMIT 30
""").df()